# Distilling Knowledge in Multiple Students Using Generative Models

In [1]:
# %tensorflow_version 1.x
# !pip install --upgrade opencv-python==3.4.2.17
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
# import os
from tensorflow.keras.datasets import fashion_mnist,mnist,cifar10
# import keras.backend as K
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Embedding,ZeroPadding2D, Input, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils
from tensorflow.keras import utils as np_utils



In [2]:
#Loading and splitting the dataset into train, validation and test
nb_classes = 10

(X_Train, y_Train), (X_test, y_test) = fashion_mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train, test_size=0.20)
# convert y_train and y_test to categorical binary values 
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)

X_train = X_train.reshape(48000, 28, 28, 1)
X_val = X_val.reshape(12000, 28, 28, 1)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')

# Normalize the values
X_train /= 255
X_val /= 255


#Creating a teacher network
input_shape = (28, 28, 1) # Input shape of each image

teacher = Sequential()
teacher.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
teacher.add(Conv2D(64, (3, 3), activation='relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))

teacher.add(Dropout(0.25)) # For reguralization

teacher.add(Flatten())
teacher.add(Dense(256, activation='relu'))
teacher.add(Dense(256, activation='relu', name="dense_1"))

teacher.add(Dropout(0.5)) # For reguralization

teacher.add(Dense(nb_classes, name = 'dense_2'))
teacher.add(Activation('softmax')) # Note that we add a normal softmax layer to begin with

teacher.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(teacher.summary())

# Train the teacher model as usual
# epochs = 30
# batch_size = 256
# teacher.fit(X_train, Y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(X_val, Y_val))


4423680/4422102 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2359552   
___________________________________________________________

In [3]:
Y_test = np_utils.to_categorical(y_test, nb_classes)
X_test = X_test.reshape(10000, 28, 28, 1)
X_test = X_test.astype('float32')
# Normalize the values
X_test /= 255
teacher.evaluate(X_test,Y_test)

313/313 [==============================] - 6s 4ms/step - loss: 2.3062 - accuracy: 0.1270


[2.305833101272583, 0.1331000030040741]

In [4]:
teacher.load_weights("Teacher_FMNIST_92.h5")

In [5]:
teacher_WO_Softmax = Model(teacher.input, teacher.get_layer('dense_1').output)
train_dense = teacher_WO_Softmax.predict(X_train)
val_dense = teacher_WO_Softmax.predict(X_val)


# 8 Students case
# ---------------------------------------------
s1Train=train_dense[:,:32]
s2Train=train_dense[:,32:64]
s3Train=train_dense[:,64:96]
s4Train=train_dense[:,96:128]
s5Train=train_dense[:,128:160]
s6Train=train_dense[:,160:192]
s7Train=train_dense[:,192:224]
s8Train=train_dense[:,224:]


## GANs' Training

In [6]:
# import np.random import random
BATCH_SIZE=32
def smooth_real_labels(y):
    return y - 0.3+(np.random.random(y.shape)*0.5)
def smooth_fake_labels(y):
    return y + (0.3 * np.random.random(y.shape))
def build_gan(gen,disc): 
    disc.trainable = False
    input= Input(shape=input_shape)
    output = gen(input)
    output2= disc(output)
    gan=Model(input,output2)

    gan.compile(Adam(lr=0.0002),loss=['binary_crossentropy','mse'],metrics=['accuracy'])

    return gan

In [7]:
def build_sdiscriminator():
    
    input2 = Input(shape=(32,),name='input')
    inp=Dense(64,use_bias=False)(input2)

    leaky_relu = LeakyReLU(alpha=0.2)(inp)
    
    conv3 = Dense(128)(leaky_relu)
    b_n = BatchNormalization()(conv3)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)
    conv4 = Dense(256)(leaky_relu)
    b_n = BatchNormalization()(conv4)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)
    conv4 = Dense(512)(leaky_relu)
    b_n = BatchNormalization()(conv4)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)
    conv4 = Dense(1024)(leaky_relu)
    b_n = BatchNormalization()(conv4)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)

    dense = Dense(1,activation='sigmoid',name='dense')(leaky_relu)
    output2=Dense(32)(leaky_relu)
    disc = Model(input2,[dense,output2])          
    disc.compile(optd,loss=['binary_crossentropy','mse'],metrics=['accuracy'])

    return disc

In [8]:
optd = Adam(lr=0.0002)
opt = Adam(lr=0.0002)

In [9]:
def build_sgenerator(name):

    student1 = Sequential()
    student1.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28, 28, 1),kernel_initializer='normal', name=name))
    student1.add(Conv2D(32, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Conv2D(16, kernel_size=(3, 3),activation='relu',kernel_initializer='normal'))
    student1.add(Conv2D(16, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Dropout(0.25)) # For reguralization
    student1.add(Flatten())
    student1.add(Dense(16, activation='relu'))
    student1.add(Dropout(0.3))
    student1.add(Dense(32,name='req'+name))

    student1.compile(opt,loss='mean_squared_error',metrics=['accuracy'])

    return student1



In [10]:
def training(generator,discriminator,gan,features,epo=20):
    BATCH_SIZE = 128
    discriminator.trainable = True
    total_size = X_train.shape[0]
    indices = np.arange(0,total_size ,BATCH_SIZE)
    all_disc_loss = []
    all_gen_loss = []
    all_class_loss=[]
    if total_size % BATCH_SIZE:
        indices = indices[:-1]
    for e in range(epo):
        
        progress_bar = Progbar(target=len(indices))
        np.random.shuffle(indices)
        epoch_gen_loss = []
        epoch_disc_loss = []
        epoch_class_loss= []
        for i,index in enumerate(indices):
        
            inputs=X_train[index:index+BATCH_SIZE]
            strain = features[index:index+BATCH_SIZE]
            y_real = np.ones((BATCH_SIZE,1))
            y_fake = np.zeros((BATCH_SIZE,1))

            #Generator Training
            fake_images = generator.predict_on_batch(inputs)

            #Disrciminator Training
            disc_real_loss1,_,disc_real_loss2,_,_= discriminator.train_on_batch(strain,[y_real,strain])
            disc_fake_loss1,_,disc_fake_loss2,_,_= discriminator.train_on_batch(fake_images,[y_fake,strain])

            #Gans Training
            discriminator.trainable = False
            gan_loss,_,gan_loss2,_,_ = gan.train_on_batch(inputs, [y_real,strain])

            discriminator.trainable = True

            disc_loss = (disc_fake_loss1 + disc_real_loss1)/2
            epoch_disc_loss.append(disc_loss)
            progress_bar.update(i+1)

            epoch_gen_loss.append((gan_loss))

        avg_epoch_disc_loss = np.array(epoch_disc_loss).mean()
        avg_epoch_gen_loss = np.array(epoch_gen_loss).mean()
        all_disc_loss.append(avg_epoch_disc_loss)
        all_gen_loss.append(avg_epoch_gen_loss)
        print("Epoch: %d | Discriminator Loss: %f | Generator Loss: %f | " % (e+1,avg_epoch_disc_loss,avg_epoch_gen_loss))

    return generator

In [11]:
discriminator1 = build_sdiscriminator()
discriminator2 = build_sdiscriminator()
discriminator3 = build_sdiscriminator()
discriminator4 = build_sdiscriminator()
discriminator5 = build_sdiscriminator()
discriminator6 = build_sdiscriminator()
discriminator7 = build_sdiscriminator()
discriminator8 = build_sdiscriminator()

In [12]:
s1=build_sgenerator("s1")
s2=build_sgenerator('s2')
s3=build_sgenerator("s3")
s4=build_sgenerator('s4')
s5=build_sgenerator("s5")
s6=build_sgenerator('s6')
s7=build_sgenerator("s7")
s8=build_sgenerator('s8')

In [13]:
gan1 = build_gan(s1,discriminator1)
gan2 = build_gan(s2,discriminator2)
gan3 = build_gan(s3,discriminator3)
gan4 = build_gan(s4,discriminator4)
gan5 = build_gan(s5,discriminator5)
gan6 = build_gan(s6,discriminator6)
gan7 = build_gan(s7,discriminator7)
gan8 = build_gan(s8,discriminator8)

In [14]:
s1 = training(s1,discriminator1,gan1,s1Train,epo=55)
s2 = training(s2,discriminator2,gan2,s2Train,epo=55)
s3 = training(s3,discriminator3,gan3,s3Train,epo=60)
s4 = training(s4,discriminator4,gan4,s4Train,epo=58)
s5 = training(s5,discriminator5,gan5,s5Train,epo=55)
s6 = training(s6,discriminator6,gan6,s6Train,epo=55)
s7 = training(s7,discriminator7,gan7,s7Train,epo=60)
s8 = training(s8,discriminator8,gan8,s8Train,epo=58)

375/375 [==============================] - 14s 34ms/step
Epoch: 1 | Discriminator Loss: 0.475441 | Generator Loss: 1.501271 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 2 | Discriminator Loss: 0.208398 | Generator Loss: 1.119375 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 3 | Discriminator Loss: 0.213619 | Generator Loss: 1.259091 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 4 | Discriminator Loss: 0.187467 | Generator Loss: 1.103767 | 
375/375 [==============================] - 13s 36ms/step
Epoch: 5 | Discriminator Loss: 0.161854 | Generator Loss: 1.020854 | 
375/375 [==============================] - 14s 36ms/step
Epoch: 6 | Discriminator Loss: 0.147970 | Generator Loss: 1.010003 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 7 | Discriminator Loss: 0.154848 | Generator Loss: 1.128355 | 
375/375 [==============================] - 14s 37ms/step
Epoch: 8 | Discriminator Loss: 0.131305 | Generator Lo

375/375 [==============================] - 14s 37ms/step
Epoch: 10 | Discriminator Loss: 0.127175 | Generator Loss: 0.712497 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 11 | Discriminator Loss: 0.129385 | Generator Loss: 0.706285 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 12 | Discriminator Loss: 0.125440 | Generator Loss: 0.652860 | 
375/375 [==============================] - 13s 36ms/step
Epoch: 13 | Discriminator Loss: 0.120057 | Generator Loss: 0.613276 | 
375/375 [==============================] - 13s 36ms/step
Epoch: 14 | Discriminator Loss: 0.114932 | Generator Loss: 0.575568 | 
375/375 [==============================] - 13s 36ms/step
Epoch: 15 | Discriminator Loss: 0.111293 | Generator Loss: 0.547689 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 16 | Discriminator Loss: 0.108748 | Generator Loss: 0.536050 | 
375/375 [==============================] - 14s 37ms/step
Epoch: 17 | Discriminator Loss: 0.106868 | Gene

375/375 [==============================] - 13s 36ms/step
Epoch: 19 | Discriminator Loss: 0.123664 | Generator Loss: 0.772926 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 20 | Discriminator Loss: 0.119746 | Generator Loss: 0.754758 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 21 | Discriminator Loss: 0.118341 | Generator Loss: 0.719564 | 
375/375 [==============================] - 14s 36ms/step
Epoch: 22 | Discriminator Loss: 0.116828 | Generator Loss: 0.681136 | 
375/375 [==============================] - 13s 36ms/step
Epoch: 23 | Discriminator Loss: 0.115070 | Generator Loss: 0.659132 | 
375/375 [==============================] - 13s 36ms/step
Epoch: 24 | Discriminator Loss: 0.113435 | Generator Loss: 0.635400 | 
375/375 [==============================] - 14s 37ms/step
Epoch: 25 | Discriminator Loss: 0.111656 | Generator Loss: 0.630238 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 26 | Discriminator Loss: 0.110012 | Gene

375/375 [==============================] - 13s 34ms/step
Epoch: 23 | Discriminator Loss: 0.097837 | Generator Loss: 0.584110 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 24 | Discriminator Loss: 0.098620 | Generator Loss: 0.577574 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 25 | Discriminator Loss: 0.100037 | Generator Loss: 0.562484 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 26 | Discriminator Loss: 0.100510 | Generator Loss: 0.552868 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 27 | Discriminator Loss: 0.099208 | Generator Loss: 0.524206 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 28 | Discriminator Loss: 0.098789 | Generator Loss: 0.509125 | 
375/375 [==============================] - 14s 36ms/step
Epoch: 29 | Discriminator Loss: 0.097636 | Generator Loss: 0.501477 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 30 | Discriminator Loss: 0.097124 | Gene

375/375 [==============================] - 13s 34ms/step
Epoch: 29 | Discriminator Loss: 0.202154 | Generator Loss: 1.104854 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 30 | Discriminator Loss: 0.159809 | Generator Loss: 1.060936 | 
375/375 [==============================] - 14s 37ms/step
Epoch: 31 | Discriminator Loss: 0.144667 | Generator Loss: 1.029670 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 32 | Discriminator Loss: 0.135176 | Generator Loss: 0.860243 | 
375/375 [==============================] - 14s 37ms/step
Epoch: 33 | Discriminator Loss: 0.128514 | Generator Loss: 0.845157 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 34 | Discriminator Loss: 0.124400 | Generator Loss: 0.798319 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 35 | Discriminator Loss: 0.121595 | Generator Loss: 0.795297 | 
375/375 [==============================] - 14s 37ms/step
Epoch: 36 | Discriminator Loss: 0.117287 | Gene

375/375 [==============================] - 13s 34ms/step
Epoch: 38 | Discriminator Loss: 0.089458 | Generator Loss: 0.447216 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 39 | Discriminator Loss: 0.088522 | Generator Loss: 0.442545 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 40 | Discriminator Loss: 0.088035 | Generator Loss: 0.439019 | 
375/375 [==============================] - 14s 36ms/step
Epoch: 41 | Discriminator Loss: 0.087375 | Generator Loss: 0.436497 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 42 | Discriminator Loss: 0.087227 | Generator Loss: 0.432273 | 
375/375 [==============================] - 13s 36ms/step
Epoch: 43 | Discriminator Loss: 0.086992 | Generator Loss: 0.429908 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 44 | Discriminator Loss: 0.086614 | Generator Loss: 0.428440 | 
375/375 [==============================] - 13s 36ms/step
Epoch: 45 | Discriminator Loss: 0.085676 | Gene

375/375 [==============================] - 13s 35ms/step
Epoch: 47 | Discriminator Loss: 0.091151 | Generator Loss: 0.413507 | 
375/375 [==============================] - 14s 36ms/step
Epoch: 48 | Discriminator Loss: 0.090709 | Generator Loss: 0.413413 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 49 | Discriminator Loss: 0.090481 | Generator Loss: 0.413767 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 50 | Discriminator Loss: 0.089958 | Generator Loss: 0.414156 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 51 | Discriminator Loss: 0.090075 | Generator Loss: 0.413910 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 52 | Discriminator Loss: 0.090652 | Generator Loss: 0.424674 | 
375/375 [==============================] - 12s 33ms/step
Epoch: 53 | Discriminator Loss: 0.090927 | Generator Loss: 0.429009 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 54 | Discriminator Loss: 0.090684 | Gene

375/375 [==============================] - 14s 37ms/step
Epoch: 51 | Discriminator Loss: 0.081783 | Generator Loss: 0.406986 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 52 | Discriminator Loss: 0.081111 | Generator Loss: 0.404078 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 53 | Discriminator Loss: 0.080258 | Generator Loss: 0.397272 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 54 | Discriminator Loss: 0.079899 | Generator Loss: 0.391192 | 
375/375 [==============================] - 14s 36ms/step
Epoch: 55 | Discriminator Loss: 0.079076 | Generator Loss: 0.388517 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 56 | Discriminator Loss: 0.078476 | Generator Loss: 0.389760 | 
375/375 [==============================] - 13s 34ms/step
Epoch: 57 | Discriminator Loss: 0.077963 | Generator Loss: 0.385043 | 
375/375 [==============================] - 13s 33ms/step
Epoch: 58 | Discriminator Loss: 0.077804 | Gene

### **8 Students**

In [28]:
o1=s1.get_layer("reqs1").output
o2=s2.get_layer("reqs2").output
o3=s3.get_layer("reqs3").output
o4=s4.get_layer("reqs4").output

o5=s5.get_layer("reqs5").output
o6=s6.get_layer("reqs6").output
o7=s7.get_layer("reqs7").output
o8=s8.get_layer("reqs8").output

output=tensorflow.keras.layers.concatenate([o1,o2,o3,o4, o5,o6,o7,o8])
print (output.shape)
output=Activation('relu')(output)
output2=Dropout(0.3)(output) # For reguralization
output3=Dense(10, activation="softmax", name="d1")(output2)
multi_model=Model([s1.get_layer("s1").input,s2.get_layer("s2").input,s3.get_layer("s3").input,s4.get_layer("s4").input,
                  s5.get_layer("s5").input, s6.get_layer("s6").input,s7.get_layer("s7").input,s8.get_layer("s8").input],output3)
my_weights=teacher.get_layer('dense_2').get_weights()
multi_model.get_layer("d1").set_weights(my_weights)

(None, 256)


In [29]:
for l in multi_model.layers[:len(multi_model.layers)-1]:
    l.trainable=False

In [30]:
multi_model.compile(loss='categorical_crossentropy',
            optimizer='nadam',
            metrics=['accuracy'])

In [31]:
batch_size = 128
mm_history=multi_model.fit([X_train,X_train,X_train,X_train,X_train,X_train,X_train,X_train], Y_train,
          batch_size=batch_size,
          epochs=100,
          verbose=1, validation_data=([X_val, X_val, X_val, X_val,X_val, X_val, X_val, X_val], Y_val))

Epoch 1/100
375/375 [==============================] - 5s 11ms/step - loss: 1.1462 - accuracy: 0.6483 - val_loss: 0.4225 - val_accuracy: 0.8479
Epoch 2/100
375/375 [==============================] - 4s 10ms/step - loss: 0.5932 - accuracy: 0.7882 - val_loss: 0.3972 - val_accuracy: 0.8538
Epoch 3/100
375/375 [==============================] - 4s 11ms/step - loss: 0.5272 - accuracy: 0.8094 - val_loss: 0.3915 - val_accuracy: 0.8528
Epoch 4/100
375/375 [==============================] - 4s 11ms/step - loss: 0.4916 - accuracy: 0.8205 - val_loss: 0.3879 - val_accuracy: 0.8546
Epoch 5/100
375/375 [==============================] - 4s 10ms/step - loss: 0.4746 - accuracy: 0.8247 - val_loss: 0.3843 - val_accuracy: 0.8552
Epoch 6/100
375/375 [==============================] - 4s 10ms/step - loss: 0.4640 - accuracy: 0.8273 - val_loss: 0.3836 - val_accuracy: 0.8550
Epoch 7/100
375/375 [==============================] - 4s 11ms/step - loss: 0.4570 - accuracy: 0.8300 - val_loss: 0.3805 - val_accuracy:

375/375 [==============================] - 4s 11ms/step - loss: 0.4455 - accuracy: 0.8364 - val_loss: 0.3797 - val_accuracy: 0.8570
Epoch 58/100
375/375 [==============================] - 4s 11ms/step - loss: 0.4416 - accuracy: 0.8362 - val_loss: 0.3784 - val_accuracy: 0.8559
Epoch 59/100
375/375 [==============================] - 4s 11ms/step - loss: 0.4365 - accuracy: 0.8369 - val_loss: 0.3776 - val_accuracy: 0.8569
Epoch 60/100
375/375 [==============================] - 4s 10ms/step - loss: 0.4366 - accuracy: 0.8364 - val_loss: 0.3812 - val_accuracy: 0.8569
Epoch 61/100
375/375 [==============================] - 4s 11ms/step - loss: 0.4473 - accuracy: 0.8336 - val_loss: 0.3781 - val_accuracy: 0.8576
Epoch 62/100
375/375 [==============================] - 4s 11ms/step - loss: 0.4436 - accuracy: 0.8344 - val_loss: 0.3808 - val_accuracy: 0.8571
Epoch 63/100
375/375 [==============================] - 4s 9ms/step - loss: 0.4383 - accuracy: 0.8372 - val_loss: 0.3774 - val_accuracy: 0.8585

In [32]:
loss, acc = multi_model.evaluate([X_test,X_test,X_test,X_test,X_test,X_test,X_test,X_test],Y_test)
loss, acc

313/313 [==============================] - 3s 8ms/step - loss: 0.3916 - accuracy: 0.8575


(0.3915756344795227, 0.8575000166893005)